In [ ]:
# import libraries
from bs4 import BeautifulSoup
import requests as req
import pickle
import re
import pandas as pd

In [ ]:
'''
Scrape data from:
1. https://www.sgcarmart.com/used_cars/info_financial.php?ID={}
2. https://www.sgcarmart.com/used_cars/info.php?ID={}

Scrape data for:
1. price (from financial table - table: {id:usedcar_price_id}, {attr:value})
2. depreciation/year
3. reg date
4. manufactured year
5. mileage
6. road tax
7. transmission
8. dereg value
9. OMV (open market value)
10. COE
11. ARF (additional refistration fees)
12. Engine cap
13. power
14. curb weight
15. number of owner
16. types
17. features
18. accessories
19. descriptions
20. warranty
21. category
22. status
'''

'\nScrape data from:\n1. https://www.sgcarmart.com/used_cars/info_financial.php?ID={}\n2. https://www.sgcarmart.com/used_cars/info.php?ID={}\n\nScrape data for:\n1. price (from financial table - table: {id:usedcar_price_id}, {attr:value})\n2. depreciation/year\n3. reg date\n4. manufactured year\n5. mileage\n6. road tax\n7. transmission\n8. dereg value\n9. OMV (open market value)\n10. COE\n11. ARF (additional refistration fees)\n12. Engine cap\n13. power\n14. curb weight\n15. number of owner\n16. types\n17. features\n18. accessories\n19. descriptions\n20. warranty\n21. category\n22. status\n'

In [ ]:
# find a spefic href tag pattern
def car_model_href(href):
    return href and re.compile(r"(^(info.php).*)").search(href)

# creating lists for records
car_id_dl_map = {}
id_list = []
dl_list = []

In [ ]:
# loop through all pages in sgcarmart used car listing
BRSR = 0
RPG = 100
for BRSR in range(0, 300, 100): #100000
      url = f"https://www.sgcarmart.com/used_cars/listing.php?BRSR={BRSR}&RPG={RPG}"
      html_text = req.get(url)
      soup = BeautifulSoup(html_text.content, "lxml")
      listings = soup.body.find('div', {'class': 'listing_searchbar_position'}).p.string.replace(" ", "")
      cleaned_listings = re.sub("\D", '', listings)
      listings_count = int(cleaned_listings)
      # car models found per page
      car_model_list = soup.body.find('div', {'id':'content'}).find('form', {'name':'searchform'}).next_sibling.next_sibling.find_all(href=car_model_href, string=True)
      for model in car_model_list:
          id = str(model).partition('ID')[2].partition('&')[0].partition('=')[-1]
          dl = str(model).partition('DL')[2].partition('"')[0].partition('=')[-1]
          if id not in car_id_dl_map:
              car_id_dl_map[id] = dl
              id_list.append(id)
              dl_list.append(dl)

In [ ]:
# printing car_id_dl_map_dictionary
print("car_id_dl_map", str(car_id_dl_map))
 
# split dictionary into keys and values
keys = car_id_dl_map.keys()
values = car_id_dl_map.values()
 
# printing keys and values separately
print("keys : ", str(keys))
print("values : ", str(values))

# convert dictionary into list
keysList = list(car_id_dl_map.keys())
valuesList = list(car_id_dl_map.values())

# printing keyslist and valueslist separately
print("keysList : ", str(keysList))
print("valuesList : ", str(valuesList))

car_id_dl_map {'1181970': '3989', '1181944': '4348', '1178900': '2138', '1173022': '3811', '1181969': '3989', '1181965': '4145', '1157888': '1194', '1178965': '3811', '1177875': '3292', '1165507': '3875', '1154849': '3292', '1181977': '3989', '1181975': '3588', '1181968': '3137', '1177448': '3792', '1174382': '3759', '1170231': '4068', '1164166': '3923', '1163668': '3175', '1149920': '2434', '1181966': '1242', '1181962': '3218', '1181961': '2563', '1181957': '3838', '1181949': '1000', '1179087': '3415', '1178973': '3811', '1173574': '3292', '1172489': '3811', '1135683': '3282', '1181959': '1013', '1181956': '3368', '1181929': '1000', '1180284': '4114', '1174415': '3037', '1171228': '1000', '1168525': '3759', '1166720': '3894', '1126416': '3759', '1101941': '3545', '1181958': '3845', '1181955': '3648', '1181954': '2007', '1181953': '2291', '1181952': '1073', '1181951': '2080', '1181948': '4178', '1181946': '3283', '1181945': '3142', '1181942': '3108', '1181937': '3283', '1181932': '2211

In [ ]:
# # define url, then grab the html table
# # to loop the url_list
# url = "https://www.sgcarmart.com/used_cars/info.php?ID=1174653"
# html_text = req.get(url)
# soup = BeautifulSoup(html_text.content, "lxml")
# print(type(soup))

<class 'bs4.BeautifulSoup'>


In [ ]:
# create empty list for each feature
price = []
depreciation = []
reg_date = []
manufactured_year = []
mileage = []
road_tax = []
transmission = []
dereg_value = []
omv = []
coe = []
arf = []
engine_cap = []
power = []
curb_weight = []
number_of_owner = []
types = []
features = []
accessories = []
descriptions = []
warranty = []
category = []
status = []

In [ ]:
# Methods to scrape each features
def get_price(soup):
    pr = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).find('tr').find_all('strong')[1]
    pr = str(pr).partition('/')[0].partition('>')[-1].partition('<')[0].strip()
    return pr

def get_depreciation(soup):
    dep = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(2)')[0].select('td:nth-of-type(2)')[0].find_all(string=re.compile("$"))
    dep = dep[0].replace(" ", "").strip().partition('/')[0]
    return dep

def get_registrationDate(soup):
    regDate = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(2)')[0].select('td:nth-of-type(4)')[0].find_all(string=True)
    reg = regDate[0].strip()
    return reg

def get_mileage(soup):
    mi = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('div:nth-of-type(1)')[0].find('div', {'class':'row_info'}).string
    mi = mi.strip()
    return mi

def get_roadTax(soup):
    roadTax = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(1)')[0].next_sibling.next_sibling.find('div', {'class':'row_info'}).string
    roadTax = roadTax.strip()
    return roadTax

def get_deregistrationValue(soup):
    dereg = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(3)')[0].find('div', {'class':'row_info'}).find(string=re.compile("$"))
    dereg = dereg.partition("as")[0].strip()
    return dereg

def get_coe(soup):
    coe = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
    coe = coe.strip()
    return coe

def get_engineCap(soup):
    eng = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
    eng = eng.strip()
    return eng

def get_curbWeight(soup):
    weight = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
    weight = weight.strip()
    return weight

def get_manufacturedYear(soup):
    year = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].find('div', {'class':'row_info'}).string
    year = year.strip()
    return year

def get_transmission(soup):
    trans = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].find('div', {'class':'eachInfo'}).next_sibling.next_sibling.find('div', {'class':'row_info'}).string
    trans = trans.strip()
    return trans

def get_omv(soup):
    omv = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(3)')[0].find('div', {'class':'row_info'}).string
    omv = omv.strip()
    return omv

def get_arf(soup):
    arf = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
    arf = arf.strip()
    return arf

def get_power(soup):
    power = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
    power = power.strip()
    return power

def get_number_of_owner(soup):
    owner = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
    owner = owner.strip()
    return owner

def get_type(soup):
    typ = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(4)')[0].find('a').string
    typ = typ.strip()
    return typ

def get_features(soup):
    f = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(5)')[0].text
    cleaned_f = ' '.join([t for t in (f.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
    return cleaned_f

def get_accessories(soup):
    a = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(6)')[0].text
    cleaned_a = ' '.join([t for t in (a.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
    return cleaned_a

def get_descriptions(soup):
    d = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(7)')[0].text
    cleaned_d = ' '.join([t for t in (d.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
    return cleaned_d

def get_warranty(soup):
    w = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(8)')[0].text
    cleaned_w = ' '.join([t for t in (w.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
    return cleaned_w

def get_category(soup):
    cat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(9)')[0].find('td').text
    cleaned_cat = ' '.join([t for t in (cat.replace('\r', '').replace('\n','').replace('Category', '').strip().split(" ")) if len(t) > 0])
    return cleaned_cat

def get_status(soup):
    stat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(10)')[0].find('td').text
    cleaned_stat = ' '.join([t for t in (stat.replace('\r', '').replace('\n','').replace('Status', '').strip().split(" ")) if len(t) > 0])
    return cleaned_stat



In [ ]:
# define a function to scrape features from SGCar Mart
def feature_scraping(url):
    
    print(url)
    
    # Get the html page
    html_text = req.get(url)
    soup = BeautifulSoup(html_text.content, "lxml")
    
    # Get the features
    sgmart_data = {}
    sgmart_data['price'] = get_price(soup)
    sgmart_data['depreciation'] = get_depreciation(soup)
    sgmart_data['registration_date'] = get_registrationDate(soup)
    sgmart_data['mileage'] = get_mileage(soup)
    sgmart_data['road_tax'] = get_roadTax(soup)
    sgmart_data['deregistration_value'] = get_deregistrationValue(soup)
    sgmart_data['coe'] = get_coe(soup)
    sgmart_data['engine_cap'] = get_engineCap(soup)
    sgmart_data['curb_weight'] = get_curbWeight(soup)
    sgmart_data['manufactured_year'] = get_manufacturedYear(soup)
    sgmart_data['trasmission'] = get_transmission(soup)
    sgmart_data['omv'] = get_omv(soup)
    sgmart_data['arf'] = get_arf(soup)
    sgmart_data['power'] = get_power(soup)
    sgmart_data['number_of_owner'] = get_number_of_owner(soup)
    sgmart_data['type'] = get_type(soup)
    sgmart_data['car_features'] = get_features(soup)
    sgmart_data['car_accessories'] = get_accessories(soup)
    sgmart_data['descriptions'] = get_descriptions(soup)
    sgmart_data['warranty'] = get_warranty(soup)
    sgmart_data['category'] = get_category(soup)
    sgmart_data['status'] = get_status(soup)
    return sgmart_data

In [ ]:
for key, value in car_id_dl_map.items():
    try:
      url_list = [f"https://www.sgcarmart.com/used_cars/info.php?ID={key}&DL={value}"]
      data = map(feature_scraping, url_list)
      print(list(data))
    except:
        pass

https://www.sgcarmart.com/used_cars/info.php?ID=1181970&DL=3989
[{'price': '$81,800', 'depreciation': '$17,880', 'registration_date': '30-Sep-2016', 'mileage': '90,000 km (13.9k /yr)', 'road_tax': '$1,210 /yr', 'deregistration_value': '$44,545', 'coe': '$57,002', 'engine_cap': '1,998 cc', 'curb_weight': '1,485 kg', 'manufactured_year': '2016', 'trasmission': 'Auto', 'omv': '$32,595', 'arf': '$37,633', 'power': '141.0 kW (189 bhp)', 'number_of_owner': '2', 'type': 'SUV', 'car_features': 'Features BMW 2.0L TwinPower turbocharged engine with 189 Bhp, 280nm of torque. 0 -100km/h in 7.4s. 7 speed auto transmission, iDrive, ABS, ESP, SRS airbags.', 'car_accessories': 'Accessories iDrive with GPS, Bluetooth, auto front seats with driver memory, contactless boot, tailgate, adjustable rear seats. Front and rear camera.', 'descriptions': 'Description Nice bidded car plate. $0 driveway, 100% loan available. Free 1 year warranty for engine and gearbox with 1 free servicing. Wear and tear done. Dri

In [ ]:
# url_list = ["https://www.sgcarmart.com/used_cars/info.php?ID=1174653"]
# data = map(feature_scraping, url_list)
# print(list(data))

https://www.sgcarmart.com/used_cars/info.php?ID=1174653
[{'price': '$51,800', 'depreciation': '$19,680', 'registration_date': '30-Jan-2015', 'mileage': '120,000 km (14.7k /yr)', 'road_tax': '$742 /yr', 'deregistration_value': '$31,010', 'coe': '$76,889', 'engine_cap': '1,598 cc', 'curb_weight': '1,410 kg', 'manufactured_year': '2014', 'trasmission': 'Auto', 'omv': '$34,553', 'arf': '$30,375', 'power': '125.0 kW (167 bhp)', 'number_of_owner': '2', 'type': 'Luxury Sedan', 'car_features': 'Features BMW 1.6L inline 4 cylinder twin power turbocharged engine producing 167bhp, 270nm of torque. 8 speed (A) Steptronic transmission. View specs of the BM View specs of the BMW 320i', 'car_accessories': 'Accessories BMW iDrive, navigation, retractable mirror, LED DRL, comfort access with keyless start, Bluetooth connectivity.', 'descriptions': 'Description Second owner only. Very good condition. All wear and tear done. Full servicing record. Highest trade in. Standard/high bank loan, balloon loan, 

In [ ]:
# from SGCar Mart - main_left table
# Left side column
# price, depreciation, reg date, mileage, road tax, dereg value, COE, engine cap, curb weight

In [ ]:
# # price
# pr = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).find('tr').find_all('strong')[1]
# price.append(str(pr).partition('/')[0].partition('>')[-1].partition('<')[0].strip())
# print(price)

['$52,800']


In [ ]:
# # depreciation
# dep = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(2)')[0].select('td:nth-of-type(2)')[0].find_all(string=re.compile("$"))
# depreciation.append(dep[0].replace(" ", "").strip().partition('/')[0])
# print(depreciation)

['$19,500']


In [ ]:
# # reg date
# regDate = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(2)')[0].select('td:nth-of-type(4)')[0].find_all(string=True)
# reg_date.append(regDate[0].strip())
# print(reg_date)

['30-Jan-2015']


In [ ]:
# # mileage
# mi = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('div:nth-of-type(1)')[0].find('div', {'class':'row_info'}).string
# mileage.append(mi.strip())
# print(mileage)

['120,000 km (14.9k /yr)']


In [ ]:
# # road tax
# roadTax = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(1)')[0].next_sibling.next_sibling.find('div', {'class':'row_info'}).string
# road_tax.append(roadTax.strip())
# print(road_tax)

['$742 /yr']


In [ ]:
# # dereg value
# dereg = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(3)')[0].find('div', {'class':'row_info'}).find(string=re.compile("$"))
# dereg_value.append(dereg.partition("as")[0].strip())
# print(dereg_value)

['$31,536']


In [ ]:
# # coe
# c = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
# coe.append(c.strip())
# print(coe)

['$76,889']


In [ ]:
# # engine cap
# eng = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
# engine_cap.append(eng.strip())
# print(engine_cap)

['1,598 cc']


In [ ]:
# # curb weight
# curb = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].find('td').select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
# curb_weight.append(curb.strip())
# print(curb_weight)

['1,410 kg']


In [ ]:
# for right side column
# manufactured year, transmission, OMV, ARF, power, No. of owners

In [ ]:
# # manufactured year
# manuYear = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(1)')[0].find('div', {'class':'row_info'}).string
# manufactured_year.append(manuYear.strip())
# print(manufactured_year)

['2014']


In [ ]:
# # transmission
# trans = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].find('div', {'class':'eachInfo'}).next_sibling.next_sibling.find('div', {'class':'row_info'}).string
# transmission.append(trans.strip())
# print(transmission)

['Auto']


In [ ]:
# # OMV
# oMV = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(3)')[0].find('div', {'class':'row_info'}).string
# omv.append(oMV.strip())
# print(omv)

['$34,553']


In [ ]:
# # ARF
# aRF = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(4)')[0].find('div', {'class':'row_info'}).string
# arf.append(aRF.strip())
# print(arf)

['$30,375']


In [ ]:
# # Power
# pOW = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(5)')[0].find('div', {'class':'row_info'}).string
# power.append(pOW.strip())
# print(power)

['125.0 kW (167 bhp)']


In [ ]:
# # number of owners
# owner = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(3)')[0].select('td:nth-of-type(2)')[0].select('div:nth-of-type(6)')[0].find('div', {'class':'row_info'}).string
# number_of_owner.append(owner.strip())
# print(number_of_owner)

['2']


In [ ]:
# # type of vehicles
# typeV = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(4)')[0].find('a').string
# types.append(typeV.strip())
# print(types)

['Luxury Sedan']


In [ ]:
# # features
# f = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(5)')[0].text
# cleaned_f = ' '.join([t for t in (f.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
# features.append(cleaned_f)
# print(features)

['Features BMW 1.6L inline 4 cylinder twin power turbocharged engine producing 167bhp, 270nm of torque. 8 speed (A) Steptronic transmission. View specs of the BM View specs of the BMW 320i']


In [ ]:
# # accessories
# a = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(6)')[0].text
# cleaned_a = ' '.join([t for t in (a.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
# accessories.append(cleaned_a)
# print(accessories)

['Accessories BMW iDrive, navigation, retractable mirror, LED DRL, comfort access with keyless start, Bluetooth connectivity.']


In [ ]:
# # descriptions
# d = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(7)')[0].text
# cleaned_d = ' '.join([t for t in (d.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
# descriptions.append(cleaned_d)
# print(descriptions)

['Description Second owner only. Very good condition. All wear and tear done. Full servicing record. Highest trade in. Standard/high bank loan, balloon loan, in house loan. 100% loan available. 1 day loan approval. Viewing & test drive today at balestier office anytime.']


In [ ]:
# # warranty
# w = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(8)')[0].text
# cleaned_w = ' '.join([t for t in (w.replace('\r', '').replace('\n','').strip().split(" ")) if len(t) > 0])
# warranty.append(cleaned_w)
# print(warranty)

['This car comes with 6-mth Sgcarmart Warranty, the best protection for your car. Ask the dealer to activate the warranty to start the coverage.$0 Repair BillsComprehensive Vehicle Inspection']


In [ ]:
# # category
# cat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(9)')[0].find('td').text
# cleaned_cat = ' '.join([t for t in (cat.replace('\r', '').replace('\n','').replace('Category', '').strip().split(" ")) if len(t) > 0])
# category.append(cleaned_cat)
# print(category)

['PARF Car, Premium Ad Car, Sgcarmart Warranty Cars']


In [ ]:
# # status
# stat = soup.body.find('div', {'id':'main_left'}).find('table', {'id':'carInfo'}).select('tr:nth-of-type(10)')[0].find('td').text
# cleaned_stat = ' '.join([t for t in (stat.replace('\r', '').replace('\n','').replace('Status', '').strip().split(" ")) if len(t) > 0])
# status.append(cleaned_stat)
# print(status)

['Available for sale. Shortlist this car to get alerted whenever the price or availability changes.']
